# TOSS: Throw Out (Your) Shell Scripts

```
import pathlib
import shutil
import subprocess
```

## Nick Timkovich

# The Incumbent: `sh`/`cmd.exe`

Unfortunately Python isn't everywhere. One of `sh` or `cmd.exe` is on nearly every development environment.

This is fine for one-offs. 
* `ls`, 
* `dir`, 
* `rm -rf /`, and so on.

But what about when we want to do a bunch of stuff again and again?

## Scripts

We can weave together commands into a file and run it using the shell.

```sh
#!/bin/sh
echo "Who are you?:"
read name
touch ${name}_was_here
```

## What if we want to do different things?

```bash
#!/bin/sh
argument=$1
if [ $argument -ne 0 ]
then
    echo "yerp"
else
    echo "nope"
fi
```

TODO MORE SIDE-BY-SIDES OF THE TRICKY LOGIC IN SH

# Shell "Features"

## One Type: Strings

Solution: use different operators to coerce and compare as other types. Like Perl.

x = 3, y = 10

```sh
# *sh
[ x -lt y ]  # true
[[ x < y ]]  # false, but bash only
```

```perl
# Perl
x lt y  # false (lt compares as strings)
x < y   # true (< compares as numbers)
```

Bash has arrays, but they are seldom used or manipulated outside of Bash-specific scripts

Sequences in shells scripts are more commonly newline- or null-delimited strings.

## Syntax Gotchas

### Expansions

```
$ touch $x

$ ls -l
total 0
-rw-r--r-- 1 nick nick    0 Aug  2 11:44 hello
-rw-r--r-- 1 nick nick    0 Aug  2 11:44 world

$ touch "$x"
total 0
-rw-r--r-- 1 nick nick 0 Aug  2 11:44  hello
-rw-r--r-- 1 nick nick 0 Aug  2 11:45 'hello world'
-rw-r--r-- 1 nick nick 0 Aug  2 11:44  world
```

Related joke command (DO NOT RUN THIS): `mv /*/`

## Syntax Gotchas

Whitespace sensitive

```text
$ x = "hello world"
x: command not found

$ x="hello world"
$ echo $x
hello world

$ [$x = "hello world"]
[hello: command not found

$ [ $x = "hello world" ]
$ # worked, but how's the output work?
```

## Returning Values

`sh` provides 8-bit unsigned return values (0-255) for status, you need to get everything else out of the output.

In [37]:
%%bash
x="hello world"

[ "$x" = "hello world" ]
echo First test : $?

[ "$x" = "hello chicago" ]
echo Second test: $?

First test : 0
Second test: 1


Note: an exit code of 0 *usually* means the command worked. In the case of the `[` executable (yes, that's a program, an alias of `test`), that it was ***true***.

## But Bash is everywhere, right?

Many scripts are cross-shell and can run in either Bash and `sh`, but some are Bash specific (`((` or `[[` is a giveaway). Bash also has versions, so if you write it locally it may not work on the production server's CentOS 6 Bash. 

* macOS 10.14 and earlier has Bash 3.x. 
* Most distros of Linux now ship Bash 4.x (2009)
* Minimal distros like Alpine won't even have it by default.
* Your local shell may not even be Bash: macOS 10.15 uses `zsh`.

If you want uniformity, you need to go for the lowest-common denominator, POSIX-compliant `sh`. 

But then you still wouldn't work on Windows...

# Great, but how do I do things in Python?

1. Implementing logic, using variables. 
    
    We won't cover this here: Hard in shell, easy in Python.


2. Manipulating the file-system (`mv`, `ls`, `cp`)
    
    `shutil`, `os`, `pathlib`


3. Launching other executables

    `subprocess` package or 3rd party ones

# Where am I?: Getting Around

When calling other executables or manipulating files, its first important to know where you are. The hidden context can be a source of confusion.

1. working directory vs. script location: important for accessing files that may be part of the script/package, or conversely that the scripts acts upon.
2. the `PATH` envvar, which shells search through for a matching executable

## The Working Directory

So where are you? The 'Working Directory' (`pwd`: print (current) working directory) is the location from which file system actions will be performed relative to.

In [36]:
%%sh
pwd

/home/nick/Code/toss


In [34]:
import os
os.getcwd()

'/home/nick/Code/toss'

This can be a gotcha if you always expect the working directory to be where your script is.

## The `PATH` envvar

Searched through for executables by some

# Subprocesses via `subprocess`

TODO

# Use Case 1: Wrapping a complicated script

A while back, I had to work with a script that would build disk images for bare-metal machines (`diskimage-builder`). This script was in-turn called by another script my team developed to set up all the customized elements, but it also made heavy use of environment variables as a way to pass in arguments.

When flitting between multiple shells across VMs, needing to re-set envvars became annoying and error-prone.

What can we do?

## Wrappers

With Python we can wrap the script, not disturbing any of the dark `sh` magic, but dramatically increasing the usability.

(Demo)

## `os.exec*`

* `execl`: Provide the arguments as a string
* `execv`: Provide the arguments as a list

* ...with `p` suffix: Searches `PATH` for the executable
* ...with `e` suffix: `env` keyword argument used to *replace* the environment variables.


# Use Case 2: Makefile Replacement

Invoke is a Python library that provides a way to execute multiple "tasks" (make would call them "directives")

Nox... (TODO)

# No Clear Winner

Undercutting my whole argument: the lowest common denominator will continue to be `/bin/sh` and compatible variants.

Howver, on your workstation or in Python app environments where you have Python installed, you may as well use it.